## Cargar paquetes

In [2]:
import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error, mean_squared_error


## Leer base de datos de Pasajeros

In [5]:
url = 'https://raw.githubusercontent.com/Chefi/Modelos-de-Control-de-Producci-n/refs/heads/main/airline-passengers.csv'

In [7]:
df = pd.read_csv(url, index_col="Month", parse_dates=True)
df

,Total_passengers
Month,
1949-01-01,112
1949-02-01,118
1949-03-01,132
1949-04-01,129
1949-05-01,121
...,...
1960-08-01,606
1960-09-01,508
1960-10-01,461


In [9]:
df.columns =["Pasajeros"]